# Multi-Class Classification - Language Classification

This notebook implements the method presented in Goldberg's [2017] book "Neural Network Methods for Natural Language Processing". It shows the steps you need to go through in order to successfully train a classifier, and it should also, so I hope, illustrate the notational differences between Goldberg and standard machine learning literature.

$NOTE$: There is no cross-validation etc. to find optimal parameters. This is simply to show how multi-class classification works. This will be part of a tutorial session and all other concepts will be explained there.

Author: Phillip Ströbel

## Getting and cleaning the data

The data consists of downloaded Wikipedia articles (see `urls.txt`) in German, English, French, Spanish, Italian and Finnish (instead of "O" in Goldberg). The data is in HTML, so we need to some preprocessing to get the text out of it. We also restrict ourselfes to the characters from a to z in the alphabet (as described in Goldberg). In this fashion, we get rid of all the Umlauts (ä, ö, ü) and all other characters with diacritics (as, e.g., the é or ç in French). Note however, that if these characters ocurring in bigrams would probably be good features. In some way, we still keep the information "special character" by not fully deleting the character, but by replacing it by the dollar sign "\$". Furthermore, we replace all punctuation marks and digits by dollar signs as well. As such, all special characters, digits, and punctuation marks are mapped to $. The space will be replaced by an underscore "\_". We then represent each langauge by 28 characters, as is suggested by Goldberg.

### Cleaning HTML
We first strip the HTML to get only the text of the Wikipedia page.

#### Get the html files

In [1]:
import re
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict

seed = np.random.seed(seed=200)  # set a seed for random, so results are reproducible

article_dict = defaultdict(lambda: defaultdict(str))

regex = r'[\n ]{2,}'
pattern = re.compile(regex)

urls = open('urls.txt', 'r').readlines()

for index, url in enumerate(urls):
    language = url[8:10]
    doc_id = 'doc_%d' % index
    html = urlopen(url.strip()).read()    
    soup = BeautifulSoup(html, 'html.parser')
    raw = soup.body.get_text()  # only get text from the text body (this excludes headers and should exclude navigation bars)
    raw = re.sub(pattern, ' ', raw)  # replace multiple breaks and spaces by only one space
    raw = re.sub(r'\n', ' ', raw)  # replace every line break with a space
    article_dict[language][doc_id] = raw.lower()  # assign each text to its language and lower all uppercase characters

### Preprocessing --> prepare the text
replace special characters and digits

In [2]:
preprocessed_dict = defaultdict(lambda: defaultdict(str))

abc = r'[a-z]'
abc_pattern = re.compile(abc)

for lang, doc in article_dict.items():
    for doc, text in doc.items():
        for char in text:
            if re.match(abc_pattern, char):
                preprocessed_dict[lang][doc] += char
            elif re.match(' ', char):
                preprocessed_dict[lang][doc] += '_'
            else:
                preprocessed_dict[lang][doc] += '$'

### Count bigrams --> Feature extraction

The distribution of bigrams will be our only feature. We could extend this by taking into account other n-grams.

In [3]:
charset = 'abcdefghijklmnopqrstuvwxyz$_'  # define the character set we want to use

In [4]:
from itertools import combinations_with_replacement, permutations

def bigrams(text):
    """
    Function to extract bigrams from text and calculate their distribution
    :param text: text string
    :return: dictionary containing bigrams as keys, and the normalised count as values
    """
    combs = combinations_with_replacement(charset, 2)
    perms = permutations(charset, 2)
    bigram_dict = dict()
    
    for comb in set(list(combs) + list(perms)):
        bigram_dict[''.join(comb)] = 0
        
    doc_length = len(text)
    
    for index in range(0, len(text)-1):
        bigram = text[index] + text[index+1]
        bigram_dict[bigram] += 1
                
    for bigram, count in bigram_dict.items():
        bigram_dict[bigram] = count/doc_length

    return bigram_dict              

### Put data into pandas dataframe
The pandas dataframe allows us to conveniently represent all the data we need in one table. So let's do this. But first we need to extract the features.

In [5]:
bigram_dict_full = defaultdict(lambda: defaultdict(dict))

for lang, doc in preprocessed_dict.items():
    for doc, text in sorted(doc.items()):
        bigram_dict = bigrams(text)
        bigram_dict_full[lang][doc] = bigram_dict

In [6]:
import pandas as pd

col_names = ['y'] + sorted(bigram_dict_full['en']['doc_0'].keys())
my_df = dict()

for col in col_names:
    my_df[col] = list()
    
df = pd.DataFrame(my_df)

for lang, doc in bigram_dict_full.items():
    for key, value in doc.items():
        df_obj = value
        df_obj['y'] = lang
        df = df.append(df_obj, ignore_index=True)
        
df.head()
        

,$$,$_,$a,$b,$c,$d,$e,$f,$g,$h,...,zq,zr,zs,zt,zu,zv,zw,zx,zy,zz
0,0.100699,0.036878,0.000291,0.000221,0.000332,0.000163,0.000239,0.000181,0.000099,0.000151,...,0.000000,0.000006,0.000006,0.0,0.000023,0.000000,0.000000,0.0,0.000000,0.000006
1,0.062188,0.028909,0.000513,0.000322,0.000520,0.000356,0.000629,0.000192,0.000185,0.000171,...,0.000007,0.000007,0.000000,0.0,0.000027,0.000000,0.000000,0.0,0.000014,0.000021
2,0.072008,0.031248,0.000444,0.000519,0.000497,0.000187,0.000340,0.000246,0.000093,0.000179,...,0.000000,0.000004,0.000000,0.0,0.000030,0.000004,0.000004,0.0,0.000011,0.000000
3,0.075458,0.035249,0.000400,0.000466,0.000453,0.000268,0.000303,0.000224,0.000312,0.000189,...,0.000000,0.000004,0.000000,0.0,0.000013,0.000000,0.000000,0.0,0.000009,0.000000
4,0.064235,0.032112,0.000344,0.000301,0.000735,0.000295,0.000476,0.000313,0.000271,0.000187,...,0.000000,0.000000,0.000006,0.0,0.000006,0.000000,0.000000,0.0,0.000000,0.000000


In [7]:
df.shape

(60, 785)

Now we split the data into the label vector \begin{equation}\mathbf{y}\end{equation} and a training data matrix \begin{equation}\mathbf{X}\end{equation}. But first, we shuffle the df and split it into a training and a test set.

Moreover, it is necessary for many machine learning tasks to standardise the data. Our aim is for each feature to be represented by a column vector in which values have zero mean and unit variance.

In [8]:
def normalise_point(datapoint, mean, std):
    """
    normalise a datapoint to zero mean and unit variance.
    :param datapoint: value as a float
    :param mean: mean of data vector x
    :param std: standard deviation of data vector x
    :return: normalised datapoint (float)
    """
    return (datapoint - mean)/std

def normalise_matrix(matrix):
    """
    normalises the data matrix
    :param matrix: input matrix
    :return: normalised matrix
    """
    train_normalised = matrix.copy()
    
    for col in matrix:
        try:
            mean = matrix[col].mean()
            std = matrix[col].std()
            for index, item in enumerate(matrix[col]):
                train_normalised.loc[index, col] = normalise_point(item, mean, std)
        except ZeroDivisionError:
            train_normalised.loc[index, col] = 0.0
        except TypeError:
            continue
    return train_normalised

In [9]:
df_norm = normalise_matrix(df)

Split the data into a train set and a test set

In [10]:
train = df_norm.sample(frac=0.9, random_state=seed)
test = df_norm.drop(train.index)

Define the different sets

In [11]:
# for training
y_train = train.y
X_train = train.drop('y', axis=1)

# for testing
y_test = test.y
X_test = test.drop('y', axis=1)

Check the shapes

In [12]:
print('Training samples shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test samples shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training samples shape:  (54, 784)
Training labels shape:  (54,)
Test samples shape:  (6, 784)
Test labels shape:  (6,)


We should binarise our labels, although libraries like sklearn can also deal with non-numeric data

In [13]:
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
lb.fit(['en', 'fr', 'de', 'it', 'es', 'fi'])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [14]:
lb.classes_

array(['de', 'en', 'es', 'fi', 'fr', 'it'], dtype='<U2')

We do this for both our training and test labels:

In [15]:
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

Labels are now one-hot encoded:

In [16]:
y_train[0]

array([0, 0, 0, 1, 0, 0])

We almost have everything now. However, we need to take care of the bias and the weight matrix. The hypothesis ŷ is given by:
\begin{equation}
\mathbf{\hat{y}}=\mathbf{x}\cdot\mathbf{W}+\mathbf{b}
\end{equation}
We can achieve this by appending 1 to each feature vector x, and the whole weight vector b to the weight matrix W. This is called the bias trick. Note that the dimensions of X_train change, and that the weight matrix W will have match the dimensions (same number of rows as X has columns).

In [17]:
bias_vector = np.ones([X_train.shape[0], 1])
X_train['bias'] = bias_vector
X_train

,$$,$_,$a,$b,$c,$d,$e,$f,$g,$h,...,zr,zs,zt,zu,zv,zw,zx,zy,zz,bias
17,0.416603,0.723617,0.415377,-0.680052,-0.972643,-0.650074,-0.396859,-0.940650,-0.862312,0.839374,...,-0.543097,-0.498310,-0.428048,-0.440917,-0.281763,-0.418054,-0.275911,-0.448826,-0.452290,1.0
5,1.117530,0.758281,-0.133773,-0.331169,-0.657555,-0.098976,-0.824004,-0.613466,-0.664787,-0.322064,...,-0.543097,-0.498310,-0.335126,-0.443883,-0.281763,-0.418054,-0.275911,0.247707,-0.426907,1.0
33,-0.324927,-0.585502,-1.236100,1.883928,-0.210752,-0.116468,-0.431933,-0.028292,0.358536,0.852603,...,0.710665,1.221814,2.202849,2.650908,-0.281763,2.054142,-0.275911,0.646775,-0.332511,1.0
48,-1.079048,0.964454,0.604140,-0.772365,-0.412629,-0.473809,-0.162890,-0.934685,-0.875055,-1.127505,...,-0.543097,-0.498310,-0.428048,-0.453567,-0.281763,-0.418054,-0.275911,-0.448826,1.417506,1.0
58,-1.161672,-1.480578,1.661224,-0.693418,0.423000,-0.342397,-0.757856,4.226816,1.265663,-1.002905,...,-0.543097,-0.498310,-0.428048,-0.412305,-0.281763,-0.418054,-0.275911,-0.448826,-0.452290,1.0
2,0.479915,-0.069664,-1.016562,-0.317783,-0.651425,-0.889001,-0.865498,-0.601891,-0.913917,-0.876159,...,-0.089304,-0.498310,-0.428048,-0.433825,0.414767,-0.404073,-0.275911,-0.052278,-0.452290,1.0
36,-0.418687,-0.630830,-0.256767,0.941374,-0.465277,0.938462,-0.617458,0.123963,0.615956,1.579467,...,2.391176,0.508124,1.453344,2.811721,1.970152,1.932453,-0.275911,1.260585,-0.405569,1.0
54,0.084962,-0.118204,0.369775,-0.562898,0.821575,-0.558084,0.326624,-0.426652,0.237723,-0.814417,...,-0.543097,0.051786,-0.428048,-0.450275,-0.281763,-0.418054,-0.275911,-0.448826,-0.375679,1.0
46,-0.868991,1.049677,0.771618,-0.119401,-0.118935,0.867179,-0.414981,-0.353783,-0.506497,-0.346314,...,0.986613,-0.498310,-0.428048,-0.438459,-0.281763,-0.418054,-0.275911,-0.003247,1.252699,1.0
51,-0.584534,-0.009742,-0.342071,-0.746197,-0.603540,-0.743240,-0.442350,-0.324134,-0.865154,-1.005620,...,-0.543097,-0.498310,-0.428048,-0.463294,-0.281763,-0.418054,2.072618,-0.448826,4.002845,1.0


In [18]:
# initialise weight matrix with small weights

np.random.seed(seed=200)

W = np.random.randn(X_train.shape[1], len(lb.classes_)) * 0.0001
#W = np.zeros([X.shape[1], len(lb.classes_)])

We see that the dimensions are right. The dot product of a specific row from X_train and the weight matrix W constitutes a forward pass and calculates the score for each class.

In [19]:
W.shape

(785, 6)

In [20]:
X_train.shape

(54, 785)

In [21]:
X_train[5:6].dot(W)

,0,1,2,3,4,5
2,-0.002759,0.001104,0.000341,-0.000792,0.003832,-0.00002


We see that the values for the highest score of the dot product is not the score of the true label. Our aim is to change this by implementing a support vector classifier.

In [22]:
X_train[5:6].dot(W).max(axis=1)

2    0.003832
dtype: float64

In [23]:
X_train[5:6].dot(W)[y_train[5:6].argmax()]

2    0.001104
Name: 1, dtype: float64

Important: we follow kind of a naive implementation. The aim is to be able to understand what is going on!

In order to quantify how good (or how bad) our weight matrix W can predict the data in our training set, we need to implement a loss function. Here we take a go at the hinge loss, which tries to predict the correct class with a margin of at least one to all other classes (or in this case, like presented in Goldberg, to the class which does not equal the true class, but which scores highest). In my understanding, this is a one-vs-one approach (true class vs. class with highest score (but doesn't equal the true class)).

In [24]:
def multiclass_hinge_loss(x, y, W, index):
    """
    Calculates the loss of a single data point by taking the prediction of the correct value and the the prediction of
    the value of next highest score, following Crammer and Singer (2001)
    :param x: sample point x as a vector
    :param y: correct label y for x as a vector
    :param W: weight matrix
    :param index: column index of data matrix X
    :return: loss
    """
    loss = 0
    y_index = y[index].argmax()
    y_value = x.dot(W)[y_index]
    y_hat_max_value = np.delete(x.dot(W), y_index).max()
    #for j in range(0, y.shape[1]):  # in case we wanted to classify against all other classes (one-vs-all) --> currently one-vs-one
        #if j == y_index:
            #continue
    loss += max(0, 1 - (y_value - y_hat_max_value))
    return loss

With matrix multiplication, we could get all the scores at once. In the following, however, we focus on an approach which takes sample by sample and calculates the loss and the gradients.

In [25]:
scores = X_train.dot(W)  # simple matrix multiplication to get all scores

In [26]:
scores

,0,1,2,3,4,5
17,-0.007368,-0.005240,-0.004246,-0.006304,0.000809,0.003532
5,-0.005714,-0.006824,0.001335,-0.000417,0.000725,-0.000727
33,0.003551,0.001367,-0.001101,-0.000361,-0.000557,0.003469
48,0.002507,0.002694,0.002059,0.000290,-0.000156,-0.002088
58,0.000668,-0.000813,0.006170,0.002906,-0.002476,0.000330
2,-0.002759,0.001104,0.000341,-0.000792,0.003832,-0.000020
36,0.000503,-0.005084,-0.002883,0.004016,0.001415,-0.000622
54,-0.001155,0.004905,0.002991,0.005849,-0.001279,0.004633
46,-0.002019,0.000097,0.001601,-0.000900,-0.001941,-0.004775
51,-0.000868,0.004477,0.001178,0.001421,0.001766,-0.004702


In [27]:
def gradient(X, y, W):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :return: Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    for index, x in enumerate(X.as_matrix()):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value))
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        dW[:, y_hat_max_index] += x.transpose()
        dW[:, y_index] -= x.transpose()
            
    return dW
    

In [28]:
def gradient_descent(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        dW = gradient(X, y, W)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [29]:
W_star = gradient_descent(X_train, y_train, W, eta=0.001, steps=1000)

### Testing
Let's test if our learned representation of the data is any good at classifying the data in the test set. Of course we need the bias in our test set as well!

In [30]:
bias_vector_test = np.ones([X_test.shape[0], 1])
X_test['bias'] = bias_vector_test

In [31]:
for index, x in enumerate(X_test.dot(W_star).as_matrix()):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

1 1
3 3
3 3
5 4
0 0
5 5


Not too bad! But Goldberg mentioned something about regularisation, so we should take this into account as well!

In [32]:
def gradient_reg(X, y, W, lam):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :param lam: reguliser lambda
    :return: Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    for index, x in enumerate(X.as_matrix()):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value)) + lam * np.linalg.norm(W, 2)
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        dW[:, y_hat_max_index] += x.transpose()
        dW[:, y_index] -= x.transpose()
        
    dW += 2 * lam * W
            
    return dW

def gradient_descent_reg(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        dW = gradient_reg(X, y, W, 10000)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [34]:
W_star_reg = gradient_descent_reg(X_train, y_train, W, eta=0.001, steps=1000)

In [35]:
for index, x in enumerate(X_test.dot(W_star_reg).as_matrix()):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

1 1
3 3
3 3
5 4
0 0
5 5


If we look at the two different weight matrices (one regularised, the other not), we notice the following:

In [36]:
W_star[0:5]

array([[ -3.62884341,   6.65219608,  -2.86095923,   9.6367692 ,
         -5.0512049 ,  -4.74783246],
       [ -8.29494248,   4.55412785,  -8.44787706,   1.96171449,
          5.50408688,   4.72306357],
       [ -7.44999196,  -5.28210705,   7.14396926,  -9.96290448,
          5.68291142,   9.86846259],
       [ 15.8677606 ,  -4.57988938,   0.8284553 ,  -5.34201712,
          3.88057968, -10.65499262],
       [ -0.66189841,  -4.25884237,   2.70457813,  -9.6260341 ,
          8.27210477,   3.57016276]])

In [37]:
W_star_reg[0:5]

array([[-7.26946916e-01,  2.83028982e+00, -4.28471754e+00,
         1.01322458e+01, -5.77413737e+00, -4.68193312e+00],
       [-7.85224904e+00,  3.59961430e+00, -7.06399831e+00,
         3.77702627e-01,  5.35758862e+00,  2.11649150e+00],
       [-7.87695494e+00, -7.31680495e+00,  3.32054570e+00,
        -8.90356121e+00,  1.99864110e+00,  1.19829328e+01],
       [ 1.75935932e+01, -5.05515238e+00,  3.13681884e+00,
        -7.77198520e+00,  6.46809785e+00, -1.23004382e+01],
       [ 2.40384248e-03, -1.69598435e+00,  2.26750250e+00,
        -1.37929826e+01,  8.76854089e+00,  3.03524196e+00]])

## By the way ...
### In scikit-learn it's much easier to implement this :-)


In [39]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, multi_class='crammer_singer', loss='hinge')
clf.fit(X_train, train.y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=0,
     tol=0.0001, verbose=0)

In [40]:
clf.predict(X_test)

array(['en', 'fi', 'fi', 'fr', 'de', 'it'], dtype=object)

In [41]:
test.y

4     en
12    fi
16    fi
26    fr
41    de
42    it
Name: y, dtype: object

We see that with our naive implementation, we do not much worse than with scikit's. scikit's implementation is of course much more elaborate and uses the vectorised operation and possibly other optimisation techniques in order to make its SVM (or SVC) better.